# Quadratic Constraints

This example demonstrates how to create and solve models with quadratic constraints (QCP/QCQP). A quadratic constraint has the form:

$$ x^T Q x + a^T x \leq b $$

In [44]:
import linopy
import numpy as np

## Setting up the Model

We'll maximize $x + 2y$ subject to a linear constraint and a quadratic constraint (a circle).

In [45]:
m = linopy.Model()

x = m.add_variables(lower=0, upper=10, name="x")
y = m.add_variables(lower=0, upper=10, name="y")

## Adding Constraints

Linear constraints work as usual:

In [46]:
m.add_constraints(x + y <= 8, name="budget")

Constraint `budget`
-------------------
+1 x + 1 y ≤ 8.0

Quadratic constraints use `add_quadratic_constraints`. Here we constrain $(x, y)$ to lie within a circle of radius 5:

In [47]:
m.add_quadratic_constraints(x * y + y * y, "<=", 25, name="circle")

QuadraticConstraint `circle`
----------------------------
1.0 x·y +1.0 y² ≤ 25.0

You can also have mixed terms like $xy$:

In [48]:
m.add_quadratic_constraints(x * y, "<=", 10, name="mixed")

QuadraticConstraint `mixed`
---------------------------
1.0 x·y ≤ 10.0

## Objective and Solve

In [49]:
m.add_objective(x + 2 * y, sense="max")

In [50]:
m.type

'QCLP'

In [51]:
m.solve(solver_name="gurobi")

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-11
Read LP format model from file /private/var/folders/2s/46_0tgfd5gq5kkpx7k42xr_w0000gn/T/linopy-problem-ylxrr3g_.lp
Reading time = 0.00 seconds
obj: 1 rows, 2 columns, 2 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0x7688eab3
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [8e+00, 8e+00]
  QRHS range       [1e+01, 2e+01]

Continuous model is non-convex -- solving as a MIP

Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 5 rows, 4 columns, 8 nonzeros
Presolved model has 1 quadratic constra

Dual values of MILP couldn't be parsed


('ok', 'optimal')

## Results

In [52]:
x.solution

<xarray.DataArray 'solution' ()> Size: 8B
array(2.5819889)

In [53]:
y.solution

<xarray.DataArray 'solution' ()> Size: 8B
array(3.87298335)

In [54]:
m.objective.value

10.327955589886445

## Inspecting Quadratic Constraints

In [55]:
m.quadratic_constraints

linopy.model.QuadraticConstraints
---------------------------------
 * circle
 * mixed

In [56]:
m.quadratic_constraints["circle"]

QuadraticConstraint `circle`
----------------------------
1.0 x·y +1.0 y² ≤ 25.0

## Multi-dimensional Quadratic Constraints

Quadratic constraints also work with coordinates, just like linear constraints:

In [57]:
m2 = linopy.Model()

# Variables with a dimension
x = m2.add_variables(lower=0, coords=[range(3)], name="x")
y = m2.add_variables(lower=0, coords=[range(3)], name="y")

# This creates 3 quadratic constraints, one for each coordinate
m2.add_quadratic_constraints(x * x * np.array([1, 1.1, 1.2]) + y * y, "<=", np.array([20, 25, 30]), name="circles")

QuadraticConstraint `circles` [dim_0: 3]:
-----------------------------------------
[0]: 1.0 x[0]² +1.0 y[0]² ≤ 20.0
[1]: 1.1 x[1]² +1.0 y[1]² ≤ 25.0
[2]: 1.2 x[2]² +1.0 y[2]² ≤ 30.0

In [58]:
m2.quadratic_constraints["circles"]

QuadraticConstraint `circles` [dim_0: 3]:
-----------------------------------------
[0]: 1.0 x[0]² +1.0 y[0]² ≤ 20.0
[1]: 1.1 x[1]² +1.0 y[1]² ≤ 25.0
[2]: 1.2 x[2]² +1.0 y[2]² ≤ 30.0